pre processing are all steps that could be done in arcgis or qgis

In [ ]:
import rasterio
from rasterio.vrt import WarpedVRT
from rasterio.enums import Resampling
import os

def resample_tiff_to_5m(input_tiff, output_tiff, resampling_method=Resampling.bilinear):
    """
    Resample a TIFF raster file to 5m resolution using specified resampling method.
    
    Parameters:
    -----------
    input_tiff : str
        Path to the input TIFF file
    output_tiff : str
        Path to the output resampled TIFF file
    resampling_method : rasterio.enums.Resampling
        Resampling method to use (default: bilinear)
        Options: nearest, bilinear, cubic, cubicspline, lanczos, average, mode, etc.
    
    Returns:
    --------
    dict : Dictionary with resampling info (output path, original resolution, new resolution)
    """
    
    with rasterio.open(input_tiff) as src:
        # Get original metadata
        meta = src.meta.copy()
        original_res = src.res[0]  # Get resolution (assuming square pixels)
        
        # Calculate scale factor (5m / original_resolution)
        scale = original_res / 5.0
        
        # Update metadata for resampling
        meta.update({
            'height': int(src.height * scale),
            'width': int(src.width * scale),
            'transform': src.transform * src.transform.scale(1.0 / scale, 1.0 / scale)
        })
        
        # Use WarpedVRT for efficient resampling
        with WarpedVRT(src, resampling=resampling_method, 
                       height=meta['height'], 
                       width=meta['width'],
                       transform=meta['transform']) as vrt:
            
            # Write resampled data to output file
            with rasterio.open(output_tiff, 'w', **meta) as dst:
                dst.write(vrt.read())
    
    return {
        'output_path': output_tiff,
        'original_resolution': original_res,
        'new_resolution': 5.0,
        'resampling_method': str(resampling_method)
    }

# Example usage:
# result = resample_tiff_to_5m('input_dem.tif', 'output_dem_5m.tif')
# print(result)